In [ ]:
!pip install openpyxl

# Notebook to create Dataset for Annotation

With two different approaches:
- Lexicon Approach
- Lexicon-Sentiment Approach

In [6]:
import pandas as pd
import numpy as np
import re
import openpyxl

In [16]:
# Load swear word dataset
df = pd.read_csv("schimpfwoerter.csv", encoding = "latin", delimiter=";")

# APPEND HATE WORDS FROM HATEBASE.ORG
hatebase = pd.read_excel("hatebaseorg.xlsx")

hatebase = hatebase["Column1"].tolist()
hatebase[:5]

['Adolf', 'Ami', 'Asylschmarotzer', 'Bazi', 'behindert']

In [8]:
len(df)

6173

In [17]:
# swear word lexicon
df

,Schimpfwort
0,1. Aa-Esser
1,2. Aal
2,3. Aalauge
3,4. Aalficker
4,5. Aalfresse
...,...
6168,6169. Zwitter
6169,6170. Zyklopenfotze
6170,6171. Zyniker
6171,6172. Zyste


In [9]:
# data cleaning
username_list = []

for index, row in df.iterrows():
    username = row['Schimpfwort'].rsplit('. ', 1)[1].rsplit(',',1)[0]
        #print(username)
    #row["username"] = username
    username_list.append(username)
df["Schimpfwort"] = username_list

In [95]:
swear_words = df["Schimpfwort"].tolist()
additional = ["no front", "cringe"]
hatebase = hatebase + swear_words + additional
len(hatebase)

6275

In [96]:
df.to_csv("schimpfwoerter_cleaned.csv")

In [97]:
# Load tweet dataset to filter after swear words
data = pd.read_csv("all_mentions_data_combined_without_retweets.csv")

/tmp/ipykernel_444/3731339884.py:1: DtypeWarning: Columns (1,2,4,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("all_mentions_data_combined_without_retweets.csv")


In [98]:
len(data)

2226216

In [83]:
#data = data.sample(10000)

In [99]:
# Filter the whole dataset after Tweets from the list
swear_word_list = []

ein_string = " ".join(hatebase)
ein_string = ein_string.lower()
ein_string = ein_string.split()
# remove swearwords, that are no real swearwords
list_to_remove = ['der','in', 'vom', 'nichts' , 'niemand', 'maske', 'art', 'menschheit', 'bild', 'null',
                 'liebe', 'zeug', 'hause', 'stumm', 'toller', 'kind', 'nase', 'langsam', 'krokodil', 
                 'zeug', 'hahn', 'pump', 'fleisch', 'gesicht', 'of', 'fahrlässig', 'kerl', 'wild',
                 'ding', 'frauenversteher', 'typ', 'bravo', 'ding', 'bär', 'tiger', 'apfel', 'dienst',
                 'dau', 'nebenmann', 'zunge', 'leidenschaft', 'currywurst']

word_list = list(set(ein_string) - set(list_to_remove))

data.text = data.text.fillna('')

# filter twitter dataset after swear words --> ~200k tweets with swear words
for index, row in data.iterrows():
    whole_tweet = row["text"]

    tweet_word = re.sub('\W+',' ', whole_tweet)
    tweet_word = tweet_word.lower()
    tweet_word = tweet_word.split()
    #print(tweet_word)
    #print(swear_words_lower)
    #match = any(x in tweet_word for x in swear_words_lower)
    match = [i for i in word_list if i in tweet_word]
    if match:
        #print(f"MATCH: {match}")
        swear_word_list.append(whole_tweet)
        #print(whole_tweet)

len(swear_word_list)

202401

In [100]:
swear_df = pd.DataFrame(data={"text": swear_word_list})
swear_df.to_csv("./swear_word_tweets.csv", sep=',',index=False)

# Create Sample and Annotation Dataset

In [14]:
# sample random 1k tweets which get annotated afterwards
swear_df = pd.read_csv("swear_word_tweets.csv")
data_sample = swear_df.sample(20000)
annotation = data_sample.sample(1000)
swear_df.head()

,text
0,1/2\nIhr #Verräter\nIhr #Eidbrecher\nIhr #Verb...
1,@SWagenknecht Weil Sie das ja auch so gut wiss...
2,@ABW67484335 @susi_lins @SWagenknecht Ach wie ...
3,"@McI0vin @Bauernopfer1 @voglerk Ich weiß ja, i..."
4,@SevimDagdelen Dağdelen erzählt immer noch ihr...


In [11]:
len(swear_df)

202401

In [15]:
pd.set_option('display.max_colwidth', None)
annotation = annotation.reset_index()
data_sample = data_sample.reset_index()
annotation

,index,text
0,165200,@Beatrix_vStorch @jamila_anna @KathrinAnna Putin macht sie alle platt... labert alle nur noch weiter blödsinn. Die nicht demokratischen Länder haben gerade alle in der Hand (Energie). Und das geht noch Jahre so... danke ihr scheiss politiker
1,142987,"@Karl_Lauterbach Auch Hitler und die DDR waren rechtlich auf der Grundlage von menschenverachtenen Schwachsinm abgesichert. Dank Capri-Sun steigt die Infektionsrate. Was für ein Rechtssystem, welches auf der Grundlage von Capri-Sun die Grundrechte missachtet! https://t.co/LjXdWpp1zm"
2,9181,@SWagenknecht Fresse halten. Sie sind eine Schande
3,84731,"@_FriedrichMerz Genau, was will den der Dichter-Robert schon wissen. Dem ist das doch egal. Schulbildung ist keine Vorraussetzung für ein Ministeramt. 10 Grüne teilen sich einen Abschluß"
4,90435,"@n_roettgen Ihre abschreckender Wirkung bezahlen Sie dann bitte aus Ihrer eigenen Tasche und nicht aus dem Säckel des Steuerzahlers, mal schauen wie weit Ihre volles Sanktionspaket noch reicht bis Ihre Taschen leer sind."
...,...,...
995,139904,"@TDuselder @VerenaRein @Da_Hiasl @R2J187 @Karl_Lauterbach Ja, wie alle anderen Querdenker."
996,34556,"@UliSpreitzer @real_Bavarian @DJanecek @EskenSaskia @Die_Gruenen Du Pfeife willst einen von uns mal in echt sehen? Könnte sein, das du danach ziemlich traumatisiert bist, bei dem, was wir täglich so sehen."
997,54254,"@NadineMaren @MarcAurel771 @Karl_Lauterbach @fdp @OlafScholz Was für ein Heuchler! Hängt seine Fahne dort hin, wo der Wind weht!\nEr ist der unglaubwürdigste Mensch (Anwalt), den es gibt!"
998,175741,@janoschdahmen @campact Anfänger. \n\nhttps://t.co/DeePgiLNpF


In [106]:
annotation.to_csv("annotation_1k.csv", sep=";", encoding="utf-8")
data_sample.to_csv("sample_dataset_20k.csv", sep=";", encoding="utf-8")